In [8]:

import json
import geopandas as gpd
import matplotlib.pyplot as plt 
from descartes import PolygonPatch
import geoplot
import geoplot.crs as gcrs
import rtree



with open('/home/ttmam/GitHub/DATAPROJECT1/datos_ini/barris-barrios.geojson') as json_file:
    json_data = json.load(json_file)


# print(json_data.keys())

barrios=[]
barrios_json = []
for i in range(len(json_data['features'])):
    barrios.append(json_data['features'][i]['geometry'])
    barrios_json.append(json_data['features'][i])


""" 
BLUE = '#6699cc'
fig = plt.figure() 
ax = fig.gca()
for i in range(len(barrios)):
    ax.add_patch(PolygonPatch(barrios[i], fc=BLUE, ec=BLUE, alpha=0.5, zorder=2 ))
ax.axis('scaled')
plt.show()
 """


with open('/home/ttmam/GitHub/DATAPROJECT1/datos_ini/zonas-verdes.geojson') as json_file2:
    json_data2 = json.load(json_file2)

# print(json_data.keys())



zonas_verdes=[]
zonas_verdes_json=[]

for i in range(len(json_data2['features'])):
    if json_data2['features'][i]['geometry'] is None:
        pass
    else:
        zonas_verdes.append(json_data2['features'][i]['geometry'])   
        zonas_verdes_json.append(json_data2['features'][i])


""" GREEN = '#00FF00'
fig = plt.figure() 
ax = fig.gca()
for i in range(len(zonas_verdes)):
    # print(i)
    ax.add_patch(PolygonPatch(zonas_verdes[i], fc=GREEN, ec=GREEN, alpha=0.5, zorder=2 ))
ax.axis('scaled')
plt.show() """



""" fig = plt.figure() 
ax = fig.gca()
for i in range(len(barrios)):
    ax.add_patch(PolygonPatch(barrios[i], fc=BLUE, ec=BLUE, alpha=0.5, zorder=2 ))
for i in range(len(zonas_verdes)):
    ax.add_patch(PolygonPatch(zonas_verdes[i], fc=GREEN, ec=GREEN, alpha=0.5, zorder=2 ))
ax.axis('scaled')
plt.show()
 """


barrios_gpd = gpd.GeoDataFrame.from_features(barrios_json)
barrios_gpd.crs = 'epsg:4326'
# barrios_gpd = barrios_gpd.to_crs({'init': 'epsg:3857'})


zonas_verdes_gpd = gpd.GeoDataFrame.from_features(zonas_verdes_json)
zonas_verdes_gpd.crs = 'epsg:4326'
# zonas_verdes_gpd = zonas_verdes_gpd.to_crs({'init': 'epsg:3857'})


""" geoplot.polyplot(zonas_verdes_gpd, projection=gcrs.AlbersEqualArea(), edgecolor='green', facecolor='green', linewidth=.3,
    figsize=(12, 8)) """
    
# print(barrios_gpd['geometry'].area)

# zonas_verdes_gpd = zonas_verdes_gpd.set_crs('epsg:3857')
zonas_verdes_gpd.crs = 'epsg:4326'
print(zonas_verdes_gpd.crs)


merged = gpd.overlay(barrios_gpd, zonas_verdes_gpd,   how='intersection')

# merged.to_csv('file_name.csv')

merged.to_file("test.shp")

print(merged)

# print(len(barrios_gpd))

# print(merged)

""" geoplot.polyplot(merged, projection=gcrs.AlbersEqualArea(), edgecolor='green', facecolor='green', linewidth=.3,
    figsize=(12, 8)) """

epsg:4326


/tmp/ipykernel_13340/3840152460.py:99: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  merged.to_file("test.shp")


ValueError: Invalid field type <class 'list'>

In [98]:
import geopandas as gpd
import geoplot
import geoplot.crs as gcrs

barrios = gpd.read_file('/home/ttmam/GitHub/DATAPROJECT1/datos_ini/barris-barrios.geojson')
barrios = barrios.set_crs('epsg:4326')

# geoplot.polyplot(barrios, projection=gcrs.AlbersEqualArea(), edgecolor='darkgrey', facecolor='lightgrey', linewidth=.3,
#     figsize=(12, 8))

zonas_verdes = gpd.read_file('/home/ttmam/GitHub/DATAPROJECT1/datos_ini/zonas-verdes.geojson')



# print(barrios)


# result = barrios.loc[barrios.within(zonas_verdes)]

# for i in range(len(barrios)):
#     areas = zonas_verdes.GeoSeries.intersection(barrios[i])

""" geoplot.polyplot(zonas_verdes, projection=gcrs.AlbersEqualArea(), edgecolor='green', facecolor='green', linewidth=.3,
    figsize=(12, 8)) """
    





merged = gpd.overlay(barrios, zonas_verdes, how='intersection')

ImportError: Spatial indexes require either `rtree` or `pygeos`. See installation instructions at https://geopandas.org/install.html

In [94]:
import geopandas as gpd
import geoplot
import geoplot.crs as gcrs

barrios = gpd.read_file('/home/ttmam/GitHub/DATAPROJECT1/datos_ini/barris-barrios.shp')

# geoplot.polyplot(barrios, projection=gcrs.AlbersEqualArea(), edgecolor='darkgrey', facecolor='lightgrey', linewidth=.3,
#     figsize=(12, 8))

zonas_verdes = gpd.read_file('/home/ttmam/GitHub/DATAPROJECT1/datos_ini/zonas-verdes.geojson')





# poly_intersections = barrios.overlay(zonas_verdes, how='intersection')